# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0033'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [7]:
spark.sql(f"""
    select  tenant_zip_code
          , vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              , tenant_zip_code
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

          ) subquery   
          
        group by 1, 2
        order by 1, 2
     
""").toPandas()

,tenant_zip_code,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,0,Janssen,1,10.00
1,0,Moderna,1,4.00
2,0,Pfizer,8,4.31
3,1,Moderna,6,5.86
4,1,Pfizer,5,5.00
5,2,Janssen,2,7.00
6,2,Moderna,5,4.50
7,2,Pfizer,19,5.50
8,3,Moderna,3,6.25
9,3,Pfizer,5,3.92


In [8]:
spark.sql(f"""
    select  tenant_zip_code
          , count(personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              , tenant_zip_code
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

          ) subquery   
          
        group by   1
        order by   1
     
""").toPandas()

,tenant_zip_code,count_of_vaccinated_patients,mean_length_of_stay
0,0,21,4.52
1,1,14,5.43
2,2,44,5.43
3,3,34,4.47
4,4,4,6.00
5,5,30,4.73
6,6,86,5.33
7,7,22,5.77
8,8,85,5.40
9,9,34,6.35


In [9]:
spark.sql(f"""
    select vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              , tenant_zip_code
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

          ) subquery   
          group by 1
          order by 1

""").toPandas()

,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,Janssen,3,8.00
1,Moderna,84,5.78
2,Pfizer,102,4.86
